In [1]:
import pandas as pd
import numpy as np
import os
from  mda_core.time import Time

In [2]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
%matplotlib inline

In [3]:
receiverType =  "Accord"
devnum = "01"
try:
    if receiverType == "Novatel":
        dataFilePath = f'./Antcom_FM_{devnum}/Processed/BESTXYZ.csv'
        dataFilePath1 = f'./Antcom_FM_{devnum}/Processed/TIME.csv'
        dataFilePath2 = f'./Antcom_FM_{devnum}/Processed/RANGE.csv'

    elif receiverType == "Accord":
        dataFilePath = f'./FM01Antenna.csv'

except: raise FileExistsError("File not found.")

In [6]:
if receiverType == "Novatel":
    pos = pd.read_csv(dataFilePath)
    time = pd.read_csv(dataFilePath1)
    range = pd.read_csv(dataFilePath2)
    pos = pos[(pos['pos type'] == 'SINGLE') & (pos['P-sol status'] == 'SOL_COMPUTED')]
    time = time[(time['utc status'] == 'VALID')]
    
    pos = pos.drop_duplicates(subset=['utc'])
    time = time.drop_duplicates(subset=['utc'])
elif receiverType == "Accord":
    pos = pd.read_csv(dataFilePath)
    pos = pos[(pos['Position Validity'] == 2)]
    pos = pos.drop_duplicates(subset=['UTC Time'])

pos.head

,UTC Time,Sats in Fix,Position Validity,P-X,P-Y,P-Z,V-X,V-Y,V-Z,PDOP,Clock bias (m),Clock drift rate (m/s),Pseudorange,Delta-Pseudorange
33,2024-04-02 12:57:44,4,2,1331575.75,6071561.26,1428666.47,0.04,-0.05,0.02,6.5,-20.10,-205.67,"[24359374.0, 22115812.0, 20505613.0, 22675500....","[-477.0, -384.0, -488.0, 226.0, 0.0, 0.0, 0.0,..."
34,2024-04-02 12:57:45,4,2,1331576.75,6071559.63,1428668.33,0.00,0.04,0.00,6.5,-0.22,-205.64,"[24359126.0, 22115655.0, 20505350.0, 22675952....","[-477.0, -384.0, -487.0, 226.0, 0.0, 0.0, 0.0,..."
35,2024-04-02 12:57:46,4,2,1331576.07,6071554.68,1428667.90,0.03,-0.03,-0.03,6.5,-3.44,-205.69,"[24358848.0, 22115475.0, 20505069.0, 22676384....","[-477.0, -384.0, -487.0, 226.0, 0.0, 0.0, 0.0,..."
36,2024-04-02 12:57:47,4,2,1331576.85,6071549.47,1428670.66,0.01,0.04,0.00,6.5,-3.60,-205.71,"[24358582.0, 22115298.0, 20504789.0, 22676820....","[-477.0, -384.0, -487.0, 226.0, 0.0, 0.0, 0.0,..."
37,2024-04-02 12:57:48,4,2,1331575.88,6071545.89,1428670.14,-0.02,-0.02,0.01,6.5,-3.75,-205.84,"[24358314.0, 22115122.0, 20504510.0, 22677254....","[-477.0, -384.0, -487.0, 226.0, 0.0, 0.0, 0.0,..."


In [8]:
epoch = []
epochtime = []
if receiverType == "Novatel":
    for i in pos['utc']:
        epoch.append(pd.Timestamp(Time(i).get_datetime()))
    data = pos
    data.index = epoch
    data.index.name = 'utc'
    # Drop unwanted columns from dataframe in addition to utc
    columns_to_keep = ['P-sol status', 'pos type', 'P-X', 'P-Y', 'P-Z', 'P-X sigma', 'P-Y sigma', 'P-Z sigma', 'V-sol status', 'vel type', 'V-X', 'V-Y', 'V-Z', 'V-X sigma', 'V-Y sigma', 'V-Z sigma']
    for col in data.columns:
        if col not in columns_to_keep:
            data = data.drop(col,axis=1)
    
    for t in time['utc']:
        epochtime.append(pd.Timestamp(Time(t).get_datetime()))
    timedata = time
    timedata.index = epochtime
    timedata.index.name = 'utc'
    columns_to_keep = ['utc year', 'utc month', 'utc day', 'utc hour', 'utc min', 'utc ms', 'utc status']
    for col in timedata.columns:
        if col not in columns_to_keep:
            timedata = timedata.drop(col,axis=1)

In [9]:
error = pd.DataFrame()
if receiverType == "Novatel":
    error['X'] = data['P-X'] 
    error['Y'] = data['P-Y'] 
    error['Z'] = data['P-Z'] 
    error['VX'] = data['V-X']
    error['VY'] = data['V-Y']
    error['VZ'] = data['V-Z']
    error['PX sigma'] = data['P-X sigma']
    error['PY sigma'] = data['P-Y sigma']
    error['PZ sigma'] = data['P-Z sigma']
    error['VX sigma'] = data['V-X sigma']
    error['VY sigma'] = data['V-Y sigma']
    error['VZ sigma'] = data['V-Z sigma']
    error['UTC hour'] = timedata['utc hour']
    error['UTC min'] = timedata['utc min']
    error['UTC sec'] = timedata['utc ms']*0.001
elif receiverType == "Accord":
    error['X'] = data['P-X'] 
    error['Y'] = data['P-Y'] 
    error['Z'] = data['P-Z'] 
    error['VX'] = data['V-X']
    error['VY'] = data['V-Y']
    error['VZ'] = data['V-Z']
error.head()

KeyError: 'P-X'

In [ ]:
def Scatter_plot(df,i,str1,str2,str3,str4):
    fig = go.Figure()
    if i == 2:
        fig.add_trace(go.Scatter(x=df.index, y=df[str1]))
        fig.add_trace(go.Scatter(x=df.index, y=df[str2]))
    elif i == 3:
        fig.add_trace(go.Scatter(x=df.index, y=df[str1]))
        fig.add_trace(go.Scatter(x=df.index, y=df[str2]))
        fig.add_trace(go.Scatter(x=df.index, y=df[str3]))
    plot_font = dict(family="Courier New, monospace", size=18, color="RebeccaPurple")
    fig.update_layout(
        font_family="Courier New",
        font_color="black",
        title_font_family="Times New Roman",
        title_font_color="red",
        legend_title_font_color="green",
        title = str4
    )
    fig.show()
    
    fig.write_html(os.path.join(f'./{receiverType}_EM',str4+'.html'))

In [ ]:
if receiverType == "Novatel":
    # Receiver Position Error
    Scatter_plot(error,3,'X','Y','Z','Receiver Position error (m)')
    # Receiver Velocity Error
    Scatter_plot(error,3,'VX','VY','VZ','Receiver Velocity error(ms)')  
    # Receiver Position Error: St Deviation
    Scatter_plot(error,3,'PX sigma','PY sigma','PZ sigma','Position Error: Receiver Computed Std Deviation (m)')
    # Receiver Velocity Error: St Deviation
    Scatter_plot(error,3,'VX sigma','VY sigma','VZ sigma','Velocity Error: Receiver Computed Std Deviation (ms)')  
    # UTC Stability
    Scatter_plot(error,3,'UTC hour','UTC min','UTC sec','UTC Stability')
elif receiverType == "Accord":
    # Receiver Position Error
    Scatter_plot(error,3,'X','Y','Z','Receiver Position error (m)')
    # Receiver Velocity Error
    Scatter_plot(error,3,'VX','VY','VZ','Receiver Velocity error (ms)')

In [ ]:
# For Receiver Position
errorpos = error[['X','Y','Z']]
errorpos['Xstd'] = errorpos['X'].std()
errorpos['Ystd'] = errorpos['Y'].std()
errorpos['Zstd'] = errorpos['Z'].std()
errorpos['Norm'] = errorpos.pow(2).sum(axis=1).pow(0.5)
errorpos['Mean'] = errorpos['Norm'].mean()
errorpos['STD'] = errorpos['Norm'].std()
errorpos['Max'] = errorpos['Norm'].max()
errorpos['Min'] = errorpos['Norm'].min()
errorpos.head()

In [ ]:
# For Receiver Velocity
errorvel = error[['VX','VY','VZ']]
errorvel['Vxstd'] = errorvel['VX'].std()
errorvel['Vystd'] = errorvel['VY'].std()
errorvel['Vzstd'] = errorvel['VZ'].std()
errorvel['Norm'] = errorvel.pow(2).sum(axis=1).pow(0.5)
errorvel['Mean'] = errorvel['Norm'].mean()
errorvel['STD'] = errorvel['Norm'].std()
errorvel['Max'] = errorvel['Norm'].max()
errorvel['Min'] = errorvel['Norm'].min()
errorvel.head()